# Introduction

- L'objectif de ce notebook:
1. Présenter les bases de données existantes en chinois
2. Appliquer les méthodes de word embedding existantes

In [3]:
import numpy as np
import re
import fasttext
import time
import multiprocessing
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
import pandas as pd
import subprocess

----
# 1.La présentation des jeux de données de texte existantes en chinois

- Nous avons trouvé un corpus chinois. Le sujet de corpus est la nouvelle. L'origine de corpus est le toolbox [THUCTC](http://thuctc.thunlp.org/, "THUCTC: 一个高效的中文文本分类工具"), une boîte d'outils pour la classification de textes chinois. Ce corpus contient 14 catégories de nouvelles. Chaque nouvelle est stockée dans un fichier .txt qui se trouve dans son propre dossier de catégorie correspondant. Voici le bilan de toutes les nouvelles.

| Catégorie | Nombre de nouvelles |
| :--------: | -----------------: |
| Sport | 131604 |
| Divertissement | 92632 |
| Maison | 32586 |
| Loterie | 7588 |
| Propriété | 20050 |
| Éducation | 41936 |
| Mode | 13368 |
| Affaire | 63086 |
| Constellation | 3578 |
| Jeu de vidéo | 24373 |
| Société | 50849 |
| Science et Technologie | 162929 |
| Action | 154398 |
| Finance et Économie | 37098 |
| **Totale** | **836075**|

- Le corpus utilisé dans ce notebook est un corpus de nouvelles de THUCTC <https://blog.csdn.net/lxg0807/article/details/52960072>. 10000 nouvelles ont été sélectionné dans chaque catégorie et ensuite ont été mis dans un seul fichier pour l'entraînement. Pour les catégories qui ne comportent pas 10000 pièces, toutes ses nouvelles ont été sélectionnées. Le jeu de données de test suit le même principe. Cependant, il ne contient pas de catégorie qui possède moins que 10000 nouvelles parce qu'ils sont tous dans le jeu de données d'apprentissage. Le [détail du code](https://blog.csdn.net/lxg0807/article/details/52776183).

- Les jeux de données utilisés dans ce notebook peuvent être téléchargés:

    [news_fasttext_train.txt](https://drive.google.com/file/d/1rF8jaifMPug2QSxYTfSJuAI6ThUEsGBi/view?usp=sharing)

    [news_fasttext_test.txt](https://drive.google.com/file/d/1psrAH5heISv3t2xuB4YKxUCvsUN6yU6C/view?usp=sharing)
    


-------------
Pour commencer, nous allons lire le jeu de données d'apprentissage et montrer quelques exemples de nouvelles.

In [3]:
with open('datasets/news_fasttext_train.txt') as f:
    f_news = f.readlines()
    
for i in range(2):
    print(f_news[i*40001])
print("Nombre de nouvelles: " + str(len(f_news)))
del(f_news)

英媒称 威廉 王子 圣诞节 前 将 宣布 订婚   　 　 中新网 11 月 8 日电   据 香港 《 文汇报 》 报道 ， 英媒称 英国 威廉 王子 和 女友 凯蒂 将 于 圣诞节 前 宣布 订婚 ， 并 于 明年 7 月 23 日拉埋 天窗 ， 结束 8 年 爱情 长跑 ， 并 计划 婚后 生儿育女 ， 组织 幸福家庭 。   　 　 报道 指 ， 28 岁 的 凯蒂 预定 与 王室 成员 一起 ， 在 诺福克郡 桑德灵 厄姆堡 欢度 圣诞节 ， 但 她 须 先 与 威廉 订婚 才 可 获邀 出席 。 消息人士 透露 ， 一切 安排 都 是 为 两人 明年 的 婚事 铺路 ， 而且 “ 进展 迅速 ” 。   　 　 两人 决定 在 温莎堡 乔治 教堂 低调 成婚 ， 这里 也 是 王储 查尔斯 和 卡米拉 2005 年 结婚 之 处 。 据悉 英女王 也 希望 在 政府 紧缩 开支 时 ， 避免 婚事 太 铺张浪费 。   　 　 传媒 本周 拍摄 到 凯蒂 父母 在 女王 位于 苏格兰 的 巴尔 莫 勒尔 堡 打猎 ， 更 获 女王 批准 在 附近 一间 王宫 住宿 ， 令 威廉 、 凯蒂 快 将 宣布 婚事 的 传闻 甚嚣尘上 。   　 　 另外 ， 女王 跟随 威廉 、 哈里 王子 和 其他 年轻 王室 成员 ， 正式 加入 大热 社交 网站 facebook ， 成立 王室 官方 页面 ， 公布 王室 活动 、 照片 、 影片 、 新闻 和 发言 。 facebook 用家 对 页面 点击 “ 赞 ” ( like ) ， 就 可 定期 收到 活动 更新 。   　 　 中新网 11 月 8 日电   据 香港 《 文汇报 》 报道 ， 英媒称 英国 威廉 王子 和 女友 凯蒂 将 于 圣诞节 前 宣布 订婚 ， 并 于 明年 7 月 23 日拉埋 天窗 ， 结束 8 年 爱情 长跑 ， 并 计划 婚后 生儿育女 ， 组织 幸福家庭 。   　 　 报道 指 ， 28 岁 的 凯蒂 预定 与 王室 成员 一起 ， 在 诺福克郡 桑德灵 厄姆堡 欢度 圣诞节 ， 但 她 须 先 与 威廉 订婚 才 可 获邀 出席 。 消息人士 透露 ， 一切 安排 都 是 为 两人 明年 的 婚事 铺路 ， 而且 “ 进展 迅速 ” 。   　 　 两人 决定 在 

----
Nous avons vu, chaque ligne du text contient une nouvelle. Tous les mots chinois ont été séparés par un espace. Les charactères liées correspondent un mot en chinois. À la fin de chaque ligne, une étiquette du sujet est marquée, avec préfix "\_\_label__".

Le nombre totale de nouvelles dan ce jeu d'apprentissage est 121155.

----
# 2. Prétraitement des données

Nous allons maintenant construire le jeu de données pour entraîner des vecteurs de Word Embedding.
- D'abord, nous allons éliminer les mots non chinois et les étiquettes. Vue que l'entraînement de vecteurs de mots est l'apprentissage non supervisé, les étiquettes ne sont pas utiles.
- Dans ce cas là, nous allons définir une méthode pour éliminer les mots non chinois et les étiquettes.

In [3]:
def filter(s):
    """
    Output only Chinese characters
    """
    filtrate = re.compile(u'[^\u4E00-\u9FA5]')
    res = filtrate.sub(r' ', s)
    return res

- Vue que la taille de données est assez grande, pour réduire l'utilisation de mémoire, nous allons faire le prétraitement de données ligne par ligne.

In [4]:
with open('datasets/news_fasttext_train.txt') as origin:
    with open('datasets/news_pure_nolabel.txt', 'w') as f:
        for line in origin:
            temp = filter(line).replace('  ', '')
            f.write("{}\n".format(temp))

- Nous avons écrit les données dans un fichier, qui peut être téléchargé via [news_pure_nolabel.txt](https://drive.google.com/file/d/1raU-oYJiZDa1U2bQcUrOuwilsQ3x49XC/view?usp=sharing)


----
# 3. Entraîner le classifier de texte par FastText

In [1]:
tic = time.time()
clf = fasttext.supervised('news_fasttext_train.txt', 'model')
toc = time.time()
print('Time used to train model: ' + str(toc - tic))

tic = time.time()
res = clf.test('news_fasttext_test.txt')
toc = time.time()
print('Time used to test model: ' + str(toc - tic))

Time used to train model: 18.09974718093872
Time used to test model: 6.489847898483276


In [4]:
print(res.precision)
print(res.recall)
print(res.nexamples)
print(clf)
help(fasttext.fasttext.FastTextModelWrapper.classifier_test)

0.9140941674970252
0.9140941674970252
103369
Help on method_descriptor:

classifier_test(...)



----
# 4. Entraîner les modèles de représentation de mots par FastText

- Il existe deux types de modèles:
    1. Skipgram modèle
    2. CBOW modèle

Les librairies **fasttext** et **gensim** permet d'entraîner les deux modèles en utilisant FastText et Word2Vec. Nous allons commencer par Fasttext

----
Nous avons utilisé les paramètres par défault pour entraîner ces modèles. Quelques paramètres importants:

| Paramètre | Value |
| --------- | ----- |
| epoch | 5 |
| vector_size | 100 |
| min_count | 5 |
| window_size | 5 |


In [6]:
# Modèle Skipgram 
tic = time.time()
model_skipgram = fasttext.skipgram('datasets/news_pure_nolabel.txt', 'model_skipgram')
toc = time.time()
print('Time used to train skipgram word representation model: ' + str(toc - tic) + "s")

print("Vocabulary size: " + str(len(model_skipgram.words))) # list of words in dictionary

Time used to train skipgram word representation model: 487.47211360931396
183312


In [7]:
# Modèle CBOW
tic = time.time()
model_cbow = fasttext.cbow('datasets/news_pure_nolabel.txt', 'model_cbow')
toc = time.time()
print('Time used to train cbow word representation model: ' + str(toc - tic) + "s")

print("Vocabulary size: " + str(len(model_cbow.words))) # list of words in dictionary

Time used to train cbow word representation model: 180.20052647590637
183312


----
# 5. Entraîner le modèle de représentation de mots par Word2Vec

Nous utilisons le module Word2Vec de package **gensim** pour faire l'apprentissage de modèle. Pareil comme FastText, nous allons entraîner Skipgram et CBOW avec les paramètres principaux comme FastText.

In [9]:
file_path = 'datasets/news_pure_nolabel.txt'

# Modèle Skipgram
tic = time.time()
model_word2vec = Word2Vec(LineSentence(file_path), workers=6, sg=1)
toc = time.time()
print("Time used to train a word2vec_skipgram word representation model: " + str(toc - tic) + "s")

# Sauvegarder le modèle et les vecteurs entraînés
model_word2vec.save('word2vec_skip.model')
model_word2vec.wv.save_word2vec_format('word2vec_skip.txt',binary = False)

Time used to train a word2vec word representation model: 767.2639379501343


In [ ]:
# Modèle CBOW
tic = time.time()
model_word2vec = Word2Vec(LineSentence(file_path), workers=6)
toc = time.time()
print("Time used to train a word2vec_cbow word representation model: " + str(toc - tic) + "s")

# Sauvegarder le modèle et les vecteurs entraînés
model_word2vec.save('word2vec_cbow.model')
model_word2vec.wv.save_word2vec_format('word2vec_cbow.txt',binary = False)

----
# 6. Entraîner le modèle de représentation de mots par GloVe

L'entraînement de modèle est fait depuis les code source de Stanford. Il n'existe pas de librairie python, donc nous allons exécuter les lignes de commande pour l'entraînement. Les paramètres principaux sont les même que ceux d'avant.

----
# 7. Évaluation des vecteurs de mots

- Après avoir entraîné le modèle, il y a trois applications de modèles usuelles:
    1. Chercher un ensemble de mots qui resemble à un mot donné
    2. Calculer le niveau de similarité de deux mots
    3. Déterminer le mot qui est différent que les autres mots dans un ensemble

In [16]:
req_count = 5
for key in model_word2vec.wv.similar_by_word('老婆', topn =100):
    if len(key[0])==3:
        req_count -= 1
        print(key[0], key[1])
        if req_count == 0:
            break;

女朋友 0.7221945524215698
男朋友 0.679075300693512
儿媳妇 0.674573540687561
老人家 0.6708532571792603
狄波拉 0.6495078802108765


- cette partie est de chercher 5 mots qui resemblent le plus à un mot donné

In [22]:
print("le niveau de similarité de deux mots: " + str(model_word2vec.wv.similarity('石头', '沙子')))
print("le niveau de similarité de deux mots: " + str(model_word2vec.wv.similarity('深圳', '广州')))
print("le niveau de similarité de deux mots: " + str(model_word2vec.wv.similarity('深圳', '深圳大学')))

le niveau de similarité de deux mots: 0.6405021458689147
le niveau de similarité de deux mots: 0.8646403886585726
le niveau de similarité de deux mots: 0.3419415565970098


- cette partie est de calculer le niveau de similarité de deux mots

In [26]:
print(model_word2vec.wv.doesnt_match(u"英国 法国 西班牙 意大利 中国".split()))

中国


- cette partie est de déterminer le mot qui est différent que les autres mots dans un ensemble